# Embedding Similarity Analysis
This notebook loads embeddings and transcript data, then allows interactive similarity calculations between any two time points.

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## 1. Configuration - Update Paths

In [27]:
# UPDATE THESE PATHS
EMBEDDINGS_NPY = "./second_level_embeddings_first_debate_with_transcript.npy"
#EMBEDDINGS_NPY = "./video_embeddings_per_second.npy"
TRANSCRIPT_CSV = "/storage/home/saichandc/qwen/transcript_by_second_first_debate.csv"

## 2. Load Embeddings

In [28]:
# Load embeddings
embeds = np.load(EMBEDDINGS_NPY)

print(f"Loaded embeddings with shape: {embeds.shape}")
print(f"\n✓ Total number of embeddings: {embeds.shape[0]}")
print(f"✓ Embedding dimension: {embeds.shape[1]}")

Loaded embeddings with shape: (210, 2048)

✓ Total number of embeddings: 210
✓ Embedding dimension: 2048

## 3. Load Transcript CSV

In [8]:
# Load transcript
df = pd.read_csv(TRANSCRIPT_CSV)

print(f"Loaded transcript CSV with shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Loaded transcript CSV with shape: (2898, 2)
Columns: ['second', 'transcript']

First few rows:

,second,transcript
0,0,Good
1,1,evening from
2,2,the Health Education Campus
3,3,of
4,4,Case Western Reserve


## 4. Auto-detect Column Names

In [9]:
# Auto-detect time and text columns
time_col = None
text_col = None

for col in df.columns:
    col_lower = col.lower()
    if col_lower in ['second', 'sec', 'time', 'seconds']:
        time_col = col
    if col_lower in ['text', 'transcript', 'content', 'speech']:
        text_col = col

if time_col is None:
    time_col = df.columns[0]
if text_col is None:
    text_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]

print(f"Using time column: '{time_col}'")
print(f"Using text column: '{text_col}'")

Using time column: 'second'
Using text column: 'transcript'

## 5. Helper Functions

In [10]:
def get_text_for_second(sec: int) -> str:
    """Get text for a specific second from transcript"""
    row = df[df[time_col] == sec]
    
    if len(row) == 0:
        return f"[No text found for second {sec}]"
    
    text = row[text_col].iloc[0]
    return str(text) if pd.notna(text) else "[Empty]"

def calculate_similarity(sec1: int, sec2: int) -> float:
    """Calculate cosine similarity between embeddings at two time points"""
    if sec1 >= len(embeds) or sec2 >= len(embeds):
        raise ValueError(f"Time index out of range. Valid range: 0-{len(embeds)-1}")
    
    # Reshape for sklearn
    emb1 = embeds[sec1].reshape(1, -1)
    emb2 = embeds[sec2].reshape(1, -1)
    
    # Calculate cosine similarity
    similarity = cosine_similarity(emb1, emb2)[0, 0]
    return similarity

def compare_seconds(sec1: int, sec2: int):
    """Compare two seconds and display similarity + text"""
    if sec1 < 0 or sec1 >= len(embeds):
        print(f"ERROR: First second must be between 0 and {len(embeds)-1}")
        return
    if sec2 < 0 or sec2 >= len(embeds):
        print(f"ERROR: Second second must be between 0 and {len(embeds)-1}")
        return
    
    # Calculate similarity
    similarity = calculate_similarity(sec1, sec2)
    
    # Get texts
    text1 = get_text_for_second(sec1)
    text2 = get_text_for_second(sec2)
    
    # Display results
    print("=" * 70)
    print(f"SIMILARITY: Second {sec1} vs Second {sec2}")
    print("=" * 70)
    print(f"\n📊 Cosine Similarity: {similarity:.4f}")
    print(f"\n📝 Text at second {sec1}:")
    print(f"   {text1}")
    print(f"\n📝 Text at second {sec2}:")
    print(f"   {text2}")
    print("=" * 70)

print("✓ Helper functions defined")

✓ Helper functions defined

## 6. Compare Two Time Points
Run this cell and modify `sec1` and `sec2` to compare different time points

In [18]:
# MODIFY THESE VALUES to compare different time points
sec1 = 1  # First second
sec2 = 62  # Second second

compare_seconds(sec1, sec2)

SIMILARITY: Second 1 vs Second 62

📊 Cosine Similarity: 0.6241

📝 Text at second 1:
   evening from

📝 Text at second 62:
   a precaution,

## 7. Quick Lookups
View text at a specific second

In [29]:
# View text at a specific second
sec = 2897  # Change this

print(f"Second {sec}: {get_text_for_second(sec)}")

Second 2897: about here.

## 8. Find Most Similar Second to a Given Time
Find which other seconds are most similar to a target second

In [14]:
def find_most_similar(target_sec: int, top_n: int = 5, exclude_self: bool = True):
    """Find the most similar seconds to a target second"""
    if target_sec >= len(embeds):
        print(f"ERROR: Second must be between 0 and {len(embeds)-1}")
        return
    
    # Get target embedding
    target_emb = embeds[target_sec].reshape(1, -1)
    
    # Calculate similarity with all embeddings
    similarities = cosine_similarity(target_emb, embeds)[0]
    
    # Get top N indices
    if exclude_self:
        similarities[target_sec] = -1  # Exclude self
    
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    
    # Display results
    print("=" * 70)
    print(f"Top {top_n} most similar seconds to second {target_sec}")
    print("=" * 70)
    print(f"\nTarget (second {target_sec}): {get_text_for_second(target_sec)}")
    print("\n" + "=" * 70)
    
    for i, idx in enumerate(top_indices, 1):
        sim = similarities[idx]
        text = get_text_for_second(idx)
        print(f"\n{i}. Second {idx} (similarity: {sim:.4f})")
        print(f"   {text}")
    print("\n" + "=" * 70)

# Example usage
target_second = 57  # Change this
find_most_similar(target_second, top_n=5)

Top 5 most similar seconds to second 57

Target (second 57): as the health security


1. Second 58 (similarity: 0.9910)
   advisor to

2. Second 59 (similarity: 0.9874)
   the Commission for

3. Second 56 (similarity: 0.8938)
   which is serving

4. Second 60 (similarity: 0.8937)
   all four debates.

5. Second 62 (similarity: 0.8591)
   a precaution,


In [16]:
def find_most_dissimilar(target_sec: int, top_n: int = 5, exclude_self: bool = True):
    """Find the most dissimilar seconds to a target second"""
    if target_sec >= len(embeds):
        print(f"ERROR: Second must be between 0 and {len(embeds)-1}")
        return
    
    # Get target embedding
    target_emb = embeds[target_sec].reshape(1, -1)
    
    # Calculate similarity with all embeddings
    similarities = cosine_similarity(target_emb, embeds)[0]
    
    # Exclude self if requested
    if exclude_self:
        similarities[target_sec] = 2  # Set to high value to exclude (similarity range is -1 to 1)
    
    # Get bottom N indices (most dissimilar = lowest similarity)
    bottom_indices = np.argsort(similarities)[:top_n]
    
    # Display results
    print("=" * 70)
    print(f"Top {top_n} most DISSIMILAR seconds to second {target_sec}")
    print("=" * 70)
    print(f"\nTarget (second {target_sec}): {get_text_for_second(target_sec)}")
    print("\n" + "=" * 70)
    
    for i, idx in enumerate(bottom_indices, 1):
        sim = similarities[idx]
        text = get_text_for_second(idx)
        print(f"\n{i}. Second {idx} (similarity: {sim:.4f})")
        print(f"   {text}")
    print("\n" + "=" * 70)

# Example usage
target_second = 1  # Change this
find_most_dissimilar(target_second, top_n=5)

Top 5 most DISSIMILAR seconds to second 1

Target (second 1): evening from


1. Second 62 (similarity: 0.6241)
   a precaution,

2. Second 61 (similarity: 0.6341)
   As

3. Second 59 (similarity: 0.6649)
   the Commission for

4. Second 57 (similarity: 0.6710)
   as the health security

5. Second 58 (similarity: 0.6786)
   advisor to


## 9. Summary Statistics

In [15]:
print("=" * 70)
print("DATASET SUMMARY")
print("=" * 70)
print(f"Total embeddings: {embeds.shape[0]}")
print(f"Embedding dimension: {embeds.shape[1]}")
print(f"Transcript entries: {len(df)}")
print(f"Valid time range: 0 to {embeds.shape[0]-1} seconds")
print("=" * 70)

DATASET SUMMARY
Total embeddings: 100
Embedding dimension: 2048
Transcript entries: 2898
Valid time range: 0 to 99 seconds